## Autograd

#### Autograd is a pytorch's automatic differentiation engine

- It automatically calculates gradient(slopes) for tensors that require it.
- Gradients tell us how to change a number to make our answer better - this is essential for training neural networks.

Autograd is like a magical notebook that tells you:
"Hey! If you move this way, you’ll get closer to balance!"
That “direction to improve” = gradient.

In [1]:
import numpy as np

In [4]:
np.zeros_like([1, 2, 3, 4, 5])

array([0, 0, 0, 0, 0])

In [5]:
# build mini autograd from scratch
class Tensor:

    def __init__(self, data, requires_grad: bool = True):
        self.data = np.array(data, dtype=np.float32)
        self.requires_grad = requires_grad
        self.grad = None
        self._backward = lambda : None # function to compute gradients
        self._prev = set() # tensors created this one

    def __repr__(self):
        return f"Tensor(data={self.data}, grad={self.grad})"

    # helper method to zero grad
    def zero_grad(self):
        self.grad = np.zeros_like(self.data)

In [23]:
# addition
def add(a, b):

    out = Tensor(a.data + b.data, requires_grad=a.requires_grad or b.requires_grad)

    def _backward_add():

        if a.requires_grad:
            if a.grad is None:
                a.grad = np.zeros_like(a.data)
            a.grad += out.grad

        if b.requires_grad:
            if b.grad is None:
                b.grad = np.zeros_like(b.data)
            b.grad += out.grad

    out._backward = _backward_add
    out._prev = {a, b}

    return out

In [24]:
# multiplication
def mul(a, b):

    out = Tensor(a.data * b.data, requires_grad=a.requires_grad or b.requires_grad)

    def _backward_mul():

        if a.requires_grad:
            if a.grad is None:
                a.grad = np.zeros_like(a.data)
            a.grad += b.data * out.grad

        if b.requires_grad:
            if b.grad is None:
                b.grad = np.zeros_like(b.data)
            b.grad += a.data * out.grad

    out._backward = _backward_mul
    out._prev = {a, b}

    return out

In [25]:
# power
def power(a, exponent):

    out = Tensor(a.data ** exponent, requires_grad=a.requires_grad)

    def _backward_power():

        if a.requires_grad:
            if a.grad is None:
                a.grad = np.zeros_like(a.data)
            a.grad += exponent * (a.data ** (exponent -1)) * out.grad

    out._backward = _backward_power
    out._prev = {a}

    return out

In [26]:
def backward(tensor, grad=None):

    if grad is None:
        grad = np.ones_like(tensor.data) # gradients of output w.r.t iteself = 1
    
    tensor.grad = grad if tensor.grad is None else tensor.grad + grad
    tensor._backward()

    for t in tensor._prev:
        if t.requires_grad:
            backward(t)

#### Test the functionalities

* y=(x**2+3x)

In [31]:
# to find -> dy / dx at x = 2
# create a tensor
x = Tensor(2.0, requires_grad=True)

# forward pass
y = add(power(x, 2), mul(Tensor(3.0), x)) # output: 10.0

# backward pass
backward(y)

print(f"y = {y.data}")
print(f"dy/dx = {x.grad}")

y = 10.0
dy/dx = 16.0


### Autograd using PyTorch

In [32]:
import torch

In [34]:
# create a tensors
x = torch.tensor(2.0, requires_grad=True)

# constant does not requires grad
c = torch.tensor(3.0, requires_grad=False)

x, c

(tensor(2., requires_grad=True), tensor(3.))

In [37]:
# forawrd pass
y = x ** 2 + c * x
print("y = ", y.item())

y =  10.0


In [ ]:
# backward pass
y.backward()

print(f"dy/dx = {x.grad.item()}")  # RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.